In [178]:
import pandas as pd
import numpy as np

ind = pd.read_csv('./data/fundamentalista/indicadores.csv')

ind.set_index(['Indicador', 'ticker'], inplace=True)

In [179]:
def preprocess_column(value):
    """
    Realiza todas as transformações necessárias em um único passo:
    1. Remove '%' e converte para fração (divisão por 100).
    2. Adiciona vírgula antes dos dois últimos dígitos, se necessário.
    3. Substitui '.' por vazio e ',' por '.' para normalizar o formato.
    4. Substitui valores sem dígitos ou inválidos por np.nan.
    """
    # Verifica se o valor é inválido (como '-') e substitui por NaN

    # Substitui valores sem dígitos por np.nan
    value = str(value)

    if not any(char.isdigit() for char in value):
        return np.nan
    
    # Remove '%', se existir
    if "%" in value:
        value = value.replace('%', '').replace('.', '').replace(',', '.')
        value = float(value) / 100
        return value

    # Adiciona vírgula se não houver ponto ou vírgula
    if any(char.isdigit() for char in value) and not ',' in value:
        value = value[:-2] + ',' + value[-2:]
    
    # Remove '.' e troca ',' por '.'
    value = value.replace('.', '').replace(',', '.')

    # Tenta converter para float
    value = float(value)
    
    return value


In [180]:
ind = ind.map(preprocess_column)

,,Atual,2023,2022,2021,2020,2019
Indicador,ticker,,,,,,
P/L,AZEV4,-1.2500,-7.2200,-3.2200,4.2800,-16.1800,-23.2800
P/RECEITA (PSR),AZEV4,1.0200,0.8500,0.6500,10.5900,28.5900,475.1200
P/VP,AZEV4,0.6700,1.1600,2.6000,7.6100,-2.6200,-3.3800
DIVIDEND YIELD (DY),AZEV4,NaN,0.0000,0.0000,0.0000,0.0000,0.0000
PAYOUT,AZEV4,NaN,0.0000,0.0000,0.0000,0.0000,0.0000
...,...,...,...,...,...,...,...
PATRIMÔNIO / ATIVOS,CEGR3,0.3200,0.3100,0.2700,0.2700,0.3500,0.2800
PASSIVOS / ATIVOS,CEGR3,0.6800,0.6900,0.7300,0.7300,0.6500,0.7200
LIQUIDEZ CORRENTE,CEGR3,0.9900,1.1400,1.0400,0.6200,0.8000,0.6500


In [190]:
indicadores_peso = {
    'CAGR LUCROS 5 ANOS': 1,           # Crescimento de lucros, maior é melhor
    'CAGR RECEITAS 5 ANOS': 1,         # Crescimento de receitas, maior é melhor
    'DIVIDEND YIELD (DY)': 1,          # Retorno de dividendos, maior é melhor
    'DÍVIDA BRUTA / PATRIMÔNIO': -1,   # Endividamento, menor é melhor
    'DÍVIDA LÍQUIDA / EBIT': -1,       # Endividamento, menor é melhor
    'DÍVIDA LÍQUIDA / EBITDA': -1,     # Endividamento, menor é melhor
    'EV/EBIT': -1,                     # Valuation, menor é melhor
    'EV/EBITDA': -1,                   # Valuation, menor é melhor
    'GIRO ATIVOS': 1,                  # Eficiência operacional, maior é melhor
    'LIQUIDEZ CORRENTE': 1,            # Liquidez, maior é melhor
    'LPA': 1,                          # Lucro por ação, maior é melhor
    'MARGEM BRUTA': 1,                 # Margem bruta, maior é melhor
    'MARGEM EBIT': 1,                  # Margem operacional, maior é melhor
    'MARGEM EBITDA': 1,                # Margem operacional, maior é melhor
    'MARGEM LÍQUIDA': 1,               # Margem líquida, maior é melhor
    'P/ATIVO': -1,                     # Valuation, menor é melhor
    'P/ATIVO CIRC LIQ': -1,            # Valuation, menor é melhor
    'P/CAP.GIRO': -1,                  # Valuation, menor é melhor
    'P/EBIT': -1,                      # Valuation, menor é melhor
    'P/EBITDA': -1,                    # Valuation, menor é melhor
    'P/L': -1,                         # Valuation, menor é melhor
    'P/RECEITA (PSR)': -1,             # Valuation, menor é melhor
    'P/VP': -1,                        # Valuation, menor é melhor
    'PASSIVOS / ATIVOS': -1,           # Endividamento, menor é melhor
    'PATRIMÔNIO / ATIVOS': 1,          # Solidez financeira, maior é melhor
    'PAYOUT': 1,                       # Distribuição de lucros, maior é melhor
    'ROA': 1,                          # Retorno sobre ativos, maior é melhor
    'ROE': 1,                          # Retorno sobre patrimônio, maior é melhor
    'ROIC': 1,                         # Retorno sobre capital investido, maior é melhor
    'VPA': 1                           # Valor patrimonial por ação, maior é melhor
}

# Converte o dicionário para uma Série para fácil alinhamento
pesos = pd.Series(indicadores_peso)

# Ajusta os valores no DataFrame com base no peso
ind_adjusted = ind.copy()  # Faz uma cópia para manter o original intacto
ind_adjusted = ind_adjusted.mul(ind_adjusted.index.get_level_values('Indicador').map(pesos), axis=0)

# Exibe o DataFrame ajustado
display(ind_adjusted)

,,Atual,2023,2022,2021,2020,2019
Indicador,ticker,,,,,,
P/L,AZEV4,1.2500,7.2200,3.2200,-4.2800,16.1800,23.2800
P/RECEITA (PSR),AZEV4,-1.0200,-0.8500,-0.6500,-10.5900,-28.5900,-475.1200
P/VP,AZEV4,-0.6700,-1.1600,-2.6000,-7.6100,2.6200,3.3800
DIVIDEND YIELD (DY),AZEV4,NaN,0.0000,0.0000,0.0000,0.0000,0.0000
PAYOUT,AZEV4,NaN,0.0000,0.0000,0.0000,0.0000,0.0000
...,...,...,...,...,...,...,...
PATRIMÔNIO / ATIVOS,CEGR3,0.3200,0.3100,0.2700,0.2700,0.3500,0.2800
PASSIVOS / ATIVOS,CEGR3,-0.6800,-0.6900,-0.7300,-0.7300,-0.6500,-0.7200
LIQUIDEZ CORRENTE,CEGR3,0.9900,1.1400,1.0400,0.6200,0.8000,0.6500


In [193]:
setores_df = pd.read_csv('./data/ativos/tickers.csv')

# Converte o DataFrame de setores para um formato indexado por 'ticker' para facilitar o mapeamento
setores_mapeados = setores_df.set_index('ticker')

# Adiciona as colunas de setor e subsetor ao DataFrame de indicadores usando o nível 'ticker' do MultiIndex
ind['setor'] = ind.index.get_level_values('ticker').map(setores_mapeados['setor'])
ind['subsetor'] = ind.index.get_level_values('ticker').map(setores_mapeados['subsetor'])

,,Atual,2023,2022,2021,2020,2019,setor,subsetor
Indicador,ticker,,,,,,,,
P/L,AZEV4,-1.2500,-7.2200,-3.2200,4.2800,-16.1800,-23.2800,Bens Industriais,Construcao E Engenharia
P/RECEITA (PSR),AZEV4,1.0200,0.8500,0.6500,10.5900,28.5900,475.1200,Bens Industriais,Construcao E Engenharia
P/VP,AZEV4,0.6700,1.1600,2.6000,7.6100,-2.6200,-3.3800,Bens Industriais,Construcao E Engenharia
DIVIDEND YIELD (DY),AZEV4,NaN,0.0000,0.0000,0.0000,0.0000,0.0000,Bens Industriais,Construcao E Engenharia
PAYOUT,AZEV4,NaN,0.0000,0.0000,0.0000,0.0000,0.0000,Bens Industriais,Construcao E Engenharia
...,...,...,...,...,...,...,...,...,...
PATRIMÔNIO / ATIVOS,CEGR3,0.3200,0.3100,0.2700,0.2700,0.3500,0.2800,Utilidade Publica,Gas
PASSIVOS / ATIVOS,CEGR3,0.6800,0.6900,0.7300,0.7300,0.6500,0.7200,Utilidade Publica,Gas
LIQUIDEZ CORRENTE,CEGR3,0.9900,1.1400,1.0400,0.6200,0.8000,0.6500,Utilidade Publica,Gas
